In [280]:
!pip install beautifulsoup4 requests nbdev streamlit streamlit_jupyter pandas

In [281]:
# |exporti
from bs4 import BeautifulSoup
import requests
import pandas as pd
import streamlit as st

In [282]:
# |exporti
base_link = "https://www.cnil.fr/fr/intelligence-artificielle/glossaire-ia?page="
pages = []
for i in range(13):
    pages.append(base_link + str(i))


In [283]:
# |exporti
alphabet_list = list(map(chr, range(97, 123)))

In [284]:
# |exporti
alphabet_obj = {}

for char in alphabet_list:
    alphabet_obj[char]= []

In [285]:
# |exporti
def word_format(word):
    obj = {}
    title = word.find("h3", class_="definition-liste-titre").a.text
    obj["title"] = title
    definition = word.find("div", class_= "definition-liste-body").text.strip()
    obj["definition"] = definition
    obj["entry"] = title[0].lower()
    link = word.find("h3", class_="definition-liste-titre").a.get("href")
    obj["link"] = link
    
    return obj


In [286]:
# |exporti
for page in pages:
    html_text = requests.get(page).text
    soup = BeautifulSoup(html_text, 'lxml')
    words = soup.find_all("div",class_="list-inner")
    for word in words:
        obj = word_format(word)
        try:
            alphabet_obj[obj["entry"]].append(obj)
        except:
            alphabet_obj[obj["entry"]] = []
            alphabet_obj[obj["entry"]].append(obj)


In [287]:

# Streamlit interface

from streamlit_jupyter import StreamlitPatcher, tqdm
StreamlitPatcher().jupyter()
import pandas as pd



In [288]:
# |exporti
entries = alphabet_obj.keys()


In [289]:
# |exporti
st.title("GLossaire IA")
st.markdown("Les données ont été extraites du site de [CNIL](%s) à fin de créer une application qui présente les entrées de façon plus intuitive de pouvoir créer des questionnaires." % pages[0])

selected_letter = st.selectbox(
    'Par quelle lettre commence le terme que vous cherchez',
    entries)



# GLossaire IA

Les données ont été extraites du site de [CNIL](https://www.cnil.fr/fr/intelligence-artificielle/glossaire-ia?page=0) à fin de créer une application qui présente les entrées de façon plus intuitive de pouvoir créer des questionnaires.

Dropdown(description='Par quelle lettre commence le terme que vous cherchez', options=('a', 'b', 'c', 'd', 'e'…

In [290]:
# |exporti
df = pd.DataFrame(alphabet_obj[selected_letter])
st.dataframe(df)

,title,definition,entry,link
0,Algorithme,Un algorithme est la description d'une suite d...,a,/fr/definition/algorithme
1,Annotation (IA),L’annotation est le procédé par lequel les don...,a,/fr/definition/annotation-ia
2,Apprentissage auto-supervisé,L’apprentissage auto-supervisé est une méthode...,a,/fr/definition/apprentissage-auto-supervise
3,Apprentissage automatique,L’apprentissage automatique (machine learning ...,a,/fr/definition/apprentissage-automatique
4,Apprentissage continu,L’apprentissage continu est la capacité d’un s...,a,/fr/definition/apprentissage-continu
5,Apprentissage fédéré,L’apprentissage fédéré est un paradigme d'appr...,a,/fr/definition/apprentissage-federe
6,Apprentissage non supervisé,L’apprentissage non supervisé est un procédé d...,a,/fr/definition/apprentissage-non-supervise
7,Apprentissage par renforcement,L’apprentissage par renforcement est un procéd...,a,/fr/definition/apprentissage-par-renforcement
8,Apprentissage par renforcement et rétroaction ...,Approche d’apprentissage par renforcement qui ...,a,/fr/definition/apprentissage-par-renforcement-...
9,Apprentissage par transfert,"En apprentissage automatique, l’apprentissage ...",a,/fr/definition/apprentissage-par-transfert


In [291]:
# |exporti
def print_(e):
    return e["title"]
word_list= list(map(print_, alphabet_obj[selected_letter]))



In [292]:
# |exporti
selected_entry = None
isLen =len(word_list) > 0
if isLen:
    selected_entry = st.selectbox(
        'Quel mot cherchez-vous?',
        word_list)
else:
    st.write("Il n'y a pas de mots correspondant à cette entrée.")

Dropdown(description='Quel mot cherchez-vous?', options=('Algorithme ', 'Annotation (IA)', 'Apprentissage auto…

In [293]:
# |exporti
def filter_cb(x):
    if x["title"] == selected_entry:
        return True
    else: 
        return False
                
if isLen and selected_entry is not None:
    selected_word = list(filter(filter_cb, alphabet_obj[selected_letter]))
    st.markdown(selected_word[0]["definition"])
    link = "https://www.cnil.fr/"+ selected_word[0]["link"]
    st.markdown(f"[{selected_entry}](%s)" % link)

Un algorithme est la description d'une suite d'étapes permettant d'obtenir un résultat à partir d'éléments fournis en entrée. Par exemple, une recette de cuisine est un algorithme permettant d'obtenir un plat à partir de ses ingrédients! Dans le monde de plus en plus numérique dans lequel nous…

[Algorithme ](https://www.cnil.fr//fr/definition/algorithme)

In [294]:
from nbdev.export import nb_export
import os
os.listdir()
os.remove('example.py')
nb_export("ai_vocabulary_web_scraping.ipynb", lib_path="./", name="app")